# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

Explanation of each input and where it comes from.

In [34]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [15]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/rce_pilot_2_02_full_spectral.pkl")

In [16]:
# CHANNEL_MAP_AND_ALL_TRIALS_DF = pd.read_pickle("./proc/full_baseline_and_trial_lfp_traces.pkl")
# MERGED_TRIALS_AND_VIDEO = pd.read_pickle("./proc/trial_SLEAP_and_metadata.pkl")

In [17]:
theta_band = (4, 12)
gamma_band = (30, 50)

In [19]:
FONTSIZE=20

In [36]:
FULL_LFP_TRACES_PKL = "{}_03_spectral_bands.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [20]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [21]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_trace', 'MD_lfp_trace', 'LH_lfp_trace',
       'BLA_lfp_trace', 'vHPC_lfp_trace',
       'mPFC_power_all_frequencies_all_windows',
       'MD_power_all_frequencies_all_windows',
       'LH_power_all_frequencies_all_windows',
       'BLA_power_all_frequencies_all_windows',
       'vHPC_power_all_frequencies_all_windows', 'power_timestamps',
       'power_calculation_frequencies',
       'BLA_LH_coherence_all_frequencies_all_windows',
       'BLA_MD_coherence_all_frequencies_all_windows',
       'BLA_mPFC_coherence_all_frequencies_all_windows',
       'BLA_vHPC_coherence_all_frequencies_all_windows',
   

## Filtering by theta and gamma band

- Filtering theta/gamma for power

In [22]:
power_columns = [col for col in LFP_SPECTRAL_DF.columns if "power" in col and "calculation" not in col and "time" not in col]

In [23]:
power_columns

['mPFC_power_all_frequencies_all_windows',
 'MD_power_all_frequencies_all_windows',
 'LH_power_all_frequencies_all_windows',
 'BLA_power_all_frequencies_all_windows',
 'vHPC_power_all_frequencies_all_windows']

In [24]:
LFP_SPECTRAL_DF["power_calculation_frequencies"].apply(lambda x: (x >= theta_band[0]) & (x <= theta_band[1])).iloc[0]

array([False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [25]:
for col in power_columns:
    brain_region_name = col.split("_")[0]
    print(brain_region_name)
    theta_power_col = "{}_power_theta".format(brain_region_name)
    gamma_power_col = "{}_power_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_power_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["power_calculation_frequencies"] >= theta_band[0]) & (x["power_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_power_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["power_calculation_frequencies"] >= gamma_band[0]) & (x["power_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

mPFC
MD
LH
BLA
vHPC


In [26]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,mPFC_power_theta,mPFC_power_gamma,MD_power_theta,MD_power_gamma,LH_power_theta,LH_power_gamma,BLA_power_theta,BLA_power_gamma,vHPC_power_theta,vHPC_power_gamma
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.0192, 0.00833, 0.00872, 0.02284, 0.03558, 0...","[0.0003676, 0.0004709, 0.0004604, 0.0003605, 0...","[0.0314, 0.0293, 0.02188, 0.02684, 0.02391, 0....","[0.0005727, 0.0003688, 0.0005994, 0.000823, 0....","[0.02385, 0.02403, 0.02197, 0.02045, 0.0185, 0...","[0.0007935, 0.000608, 0.000549, 0.000664, 0.00...","[0.02734, 0.02118, 0.01955, 0.02567, 0.02525, ...","[0.001674, 0.001781, 0.001711, 0.001264, 0.001...","[0.01069, 0.02905, 0.05573, 0.0284, 0.0164, 0....","[0.000385, 0.0004468, 0.000421, 0.0004888, 0.0..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.0192, 0.00833, 0.00872, 0.02284, 0.03558, 0...","[0.0003676, 0.0004709, 0.0004604, 0.0003605, 0...","[0.0314, 0.0293, 0.02188, 0.02684, 0.02391, 0....","[0.0005727, 0.0003688, 0.0005994, 0.000823, 0....","[0.02385, 0.02403, 0.02197, 0.02045, 0.0185, 0...","[0.0007935, 0.000608, 0.000549, 0.000664, 0.00...","[0.02734, 0.02118, 0.01955, 0.02567, 0.02525, ...","[0.001674, 0.001781, 0.001711, 0.001264, 0.001...","[0.01069, 0.02905, 0.05573, 0.0284, 0.0164, 0....","[0.000385, 0.0004468, 0.000421, 0.0004888, 0.0..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.014854, 0.01645, 0.03424, 0.0245, 0.02141, ...","[0.0017395, 0.002783, 0.00245, 0.00219, 0.0022...","[0.01321, 0.01384, 0.02313, 0.0213, 0.01484, 0...","[0.000718, 0.000562, 0.00046, 0.0005183, 0.000...","[0.01522, 0.01698, 0.02643, 0.02547, 0.01753, ...","[0.0009117, 0.0006933, 0.0006523, 0.0008574, 0...","[0.013466, 0.0199, 0.0302, 0.02333, 0.02101, 0...","[0.00339, 0.003328, 0.003246, 0.003057, 0.0025...","[0.02547, 0.02805, 0.04492, 0.0433, 0.03625, 0...","[0.00303, 0.003956, 0.002377, 0.002512, 0.0022..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.014854, 0.01645, 0.03424, 0.0245, 0.02141, ...","[0.0017395, 0.002783, 0.00245, 0.00219, 0.0022...","[0.01321, 0.01384, 0.02313, 0.0213, 0.01484, 0...","[0.000718, 0.000562, 0.00046, 0.0005183, 0.000...","[0.01522, 0.01698, 0.02643, 0.02547, 0.01753, ...","[0.0009117, 0.0006933, 0.0006523, 0.0008574, 0...","[0.013466, 0.0199, 0.0302, 0.02333, 0.02101, 0...","[0.00339, 0.003328, 0.003246, 0.003057, 0.0025...","[0.02547, 0.02805, 0.04492, 0.0433, 0.03625, 0...","[0.00303, 0.003956, 0.002377, 0.002512, 0.0022..."
4,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405]

- Filtering theta/gamma for coherence

In [28]:
coherence_columns = [col for col in LFP_SPECTRAL_DF.columns if "coherence" in col and "calculation" not in col and "time" not in col]

In [29]:
coherence_columns

['BLA_LH_coherence_all_frequencies_all_windows',
 'BLA_MD_coherence_all_frequencies_all_windows',
 'BLA_mPFC_coherence_all_frequencies_all_windows',
 'BLA_vHPC_coherence_all_frequencies_all_windows',
 'LH_MD_coherence_all_frequencies_all_windows',
 'LH_mPFC_coherence_all_frequencies_all_windows',
 'LH_vHPC_coherence_all_frequencies_all_windows',
 'MD_mPFC_coherence_all_frequencies_all_windows',
 'MD_vHPC_coherence_all_frequencies_all_windows',
 'mPFC_vHPC_coherence_all_frequencies_all_windows']

In [30]:
for col in coherence_columns:
    brain_region_name = "_".join(col.split("_")[:2])
    print(brain_region_name)
    theta_coherence_col = "{}_coherence_theta".format(brain_region_name)
    gamma_coherence_col = "{}_coherence_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_coherence_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["coherence_calculation_frequencies"] >= theta_band[0]) & (x["coherence_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_coherence_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["coherence_calculation_frequencies"] >= gamma_band[0]) & (x["coherence_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

BLA_LH
BLA_MD
BLA_mPFC
BLA_vHPC
LH_MD
LH_mPFC
LH_vHPC
MD_mPFC
MD_vHPC
mPFC_vHPC


- Filtering theta/gamma for granger

In [31]:
granger_columns = [col for col in LFP_SPECTRAL_DF.columns if "granger" in col and "calculation" not in col and "time" not in col]

In [32]:
granger_columns

['BLA_LH_granger_all_frequencies_all_windows',
 'LH_BLA_granger_all_frequencies_all_windows',
 'BLA_MD_granger_all_frequencies_all_windows',
 'MD_BLA_granger_all_frequencies_all_windows',
 'BLA_mPFC_granger_all_frequencies_all_windows',
 'mPFC_BLA_granger_all_frequencies_all_windows',
 'BLA_vHPC_granger_all_frequencies_all_windows',
 'vHPC_BLA_granger_all_frequencies_all_windows',
 'LH_MD_granger_all_frequencies_all_windows',
 'MD_LH_granger_all_frequencies_all_windows',
 'LH_mPFC_granger_all_frequencies_all_windows',
 'mPFC_LH_granger_all_frequencies_all_windows',
 'LH_vHPC_granger_all_frequencies_all_windows',
 'vHPC_LH_granger_all_frequencies_all_windows',
 'MD_mPFC_granger_all_frequencies_all_windows',
 'mPFC_MD_granger_all_frequencies_all_windows',
 'MD_vHPC_granger_all_frequencies_all_windows',
 'vHPC_MD_granger_all_frequencies_all_windows',
 'mPFC_vHPC_granger_all_frequencies_all_windows',
 'vHPC_mPFC_granger_all_frequencies_all_windows']

In [33]:
for col in granger_columns:
    brain_region_name = "-to-".join(col.split("_")[:2])
    print(brain_region_name)
    theta_granger_col = "{}_granger_theta".format(brain_region_name)
    gamma_granger_col = "{}_granger_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_granger_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["granger_calculation_frequencies"] >= theta_band[0]) & (x["granger_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_granger_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["granger_calculation_frequencies"] >= gamma_band[0]) & (x["granger_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

BLA-to-LH
LH-to-BLA
BLA-to-MD
MD-to-BLA
BLA-to-mPFC
mPFC-to-BLA
BLA-to-vHPC
vHPC-to-BLA
LH-to-MD
MD-to-LH
LH-to-mPFC
mPFC-to-LH
LH-to-vHPC
vHPC-to-LH
MD-to-mPFC
mPFC-to-MD
MD-to-vHPC
vHPC-to-MD
mPFC-to-vHPC
vHPC-to-mPFC


In [39]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,mPFC-to-MD_granger_theta,mPFC-to-MD_granger_gamma,MD-to-vHPC_granger_theta,MD-to-vHPC_granger_gamma,vHPC-to-MD_granger_theta,vHPC-to-MD_granger_gamma,mPFC-to-vHPC_granger_theta,mPFC-to-vHPC_granger_gamma,vHPC-to-mPFC_granger_theta,vHPC-to-mPFC_granger_gamma
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.619, 0.2048, 0.416, 0.212, 0.0773, 0.3818, ...","[0.433, 0.1775, 0.1467, 0.3218, 0.1627, 0.14, ...","[0.2556, 0.561, 0.5347, 0.595, 0.6245, 0.2915,...","[0.2942, 0.2078, 0.1791, 0.2074, 0.6104, 0.416...","[0.7363, 0.623, 0.916, 0.1919, 0.4854, 0.291, ...","[0.3962, 0.3162, 0.3918, 0.3738, 0.407, 0.2546...","[0.294, 0.3416, 0.3147, 0.1201, 0.1798, 0.3499...","[0.3696, 0.4702, 0.3848, 0.2676, 0.1721, 0.214...","[0.3733, 0.3875, 0.4814, 0.0788, 0.3303, 0.236...","[0.2612, 0.1371, 0.11597, 0.5317, 0.3992, 0.31..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.619, 0.2048, 0.416, 0.212, 0.0773, 0.3818, ...","[0.433, 0.1775, 0.1467, 0.3218, 0.1627, 0.14, ...","[0.2556, 0.561, 0.5347, 0.595, 0.6245, 0.2915,...","[0.2942, 0.2078, 0.1791, 0.2074, 0.6104, 0.416...","[0.7363, 0.623, 0.916, 0.1919, 0.4854, 0.291, ...","[0.3962, 0.3162, 0.3918, 0.3738, 0.407, 0.2546...","[0.294, 0.3416, 0.3147, 0.1201, 0.1798, 0.3499...","[0.3696, 0.4702, 0.3848, 0.2676, 0.1721, 0.214...","[0.3733, 0.3875, 0.4814, 0.0788, 0.3303, 0.236...","[0.2612, 0.1371, 0.11597, 0.5317, 0.3992, 0.31..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.4675, 0.2383, 0.343, 0.0534, 0.2014, 0.1708...","[0.374, 0.2722, 0.2888, 0.3542, 0.4832, 0.2039...","[0.12067, 0.1519, 0.1978, 0.4722, 0.725, 0.346...","[0.2092, 0.2322, 0.57, 0.3848, 0.4849, 0.28, 0...","[0.824, 0.8486, 0.4873, 0.6494, 0.881, 0.4429,...","[0.2085, 0.2018, 0.2346, 0.4075, 0.25, 0.353, ...","[0.329, 0.2783, 0.3796, 0.3135, 0.18, 0.2908, ...","[0.343, 0.382, 0.6567, 0.3767, 0.4895, 0.4211,...","[0.3115, 0.345, 1.015, 1.294, 0.367, 0.648, 0....","[0.3955, 0.2693, 0.2559, 0.3315, 0.2373, 0.586..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.4675, 0.2383, 0.343, 0.0534, 0.2014, 0.1708...","[0.374, 0.2722, 0.2888, 0.3542, 0.4832, 0.2039...","[0.12067, 0.1519, 0.1978, 0.4722, 0.725, 0.346...","[0.2092, 0.2322, 0.57, 0.3848, 0.4849, 0.28, 0...","[0.824, 0.8486, 0.4873, 0.6494, 0.881, 0.4429,...","[0.2085, 0.2018, 0.2346, 0.4075, 0.25, 0.353, ...","[0.329, 0.2783, 0.3796, 0.3135, 0.18, 0.2908, ...","[0.343, 0.382, 0.6567, 0.3767, 0.4895, 0.4211,...","[0.3115, 0.345, 1.015, 1.294, 0.367, 0.648, 0....","[0.3955, 0.2693, 0.2559, 0.3315, 0.2373, 0.586..."
4,2,20230612_112630_standard_com

In [38]:
LFP_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))